In [1]:
import os

Variaveis globais necessárias para funçoes

In [2]:
datasetTxt = "Datasets/Dataset-ClassTT_01_tiny_File.txt"


Parse do dataset de horários, retornando um dicionário com todas as seções.

    Retorna:
        dict: Dicionário com as seguintes chaves:
            - 'head': dict com configurações gerais
            - 'cc': dict {turma: [cursos]}
            - 'olw': list de cursos com apenas 1 aula/semana
            - 'dsd': dict {professor: [cursos]}
            - 'tr': dict {professor: [slots indisponíveis]}
            - 'rr': dict {curso: sala}
            - 'oc': dict {curso: índice_aula_online}

In [3]:
def parse_dataset(datasetTxt):
    lines = datasetTxt.strip().split('\n')
    data = {
        'head': {},
        'cc': {},
        'olw': [],
        'dsd': {},
        'tr': {},
        'rr': {},
        'oc': {}
    }

    current_section = None

    for line in lines:
        line = line.strip()

        # Ignorar linhas vazias
        if not line:
            continue

        # Detectar cabeçalhos de seção
        if line.startswith('#'):
            section_name = line.split()[0][1:]  # Remove o '#'
            current_section = section_name
            continue

        # Ignorar comentários
        if line.startswith('—') or line.startswith('--'):
            continue

        # Processar dados baseado na seção atual
        if current_section == 'cc':
            parts = line.split()
            if parts:
                class_id = parts[0]
                courses = parts[1:]
                data['cc'][class_id] = courses

        elif current_section == 'olw':
            # Esta seção parece estar vazia no exemplo
            if line.strip():
                data['olw'].append(line.strip())

        elif current_section == 'dsd':
            parts = line.split()
            if parts:
                teacher = parts[0]
                courses = parts[1:]
                data['dsd'][teacher] = courses

        elif current_section == 'tr':
            parts = line.split()
            if parts:
                teacher = parts[0]
                slots = [int(s) for s in parts[1:]]
                data['tr'][teacher] = slots

        elif current_section == 'rr':
            parts = line.split()
            if len(parts) >= 2:
                course = parts[0]
                room = parts[1]
                data['rr'][course] = room

        elif current_section == 'oc':
            parts = line.split()
            if len(parts) >= 2:
                course = parts[0]
                week_index = int(parts[1])
                data['oc'][course] = week_index

    return data

Lê o ficheiro do dataset.

In [4]:
def load_dataset_from_file(datasetTxt):
    try:
        with open(datasetTxt, 'r', encoding='utf-8') as f:
            dataset_text = f.read()
        return parse_dataset(dataset_text)
    except FileNotFoundError:
        print(f"Erro: Ficheiro '{datasetTxt}' não encontrado!")
        return None
    except Exception as e:
        print(f"Erro ao ler ficheiro: {e}")
        return None

## Testes

In [5]:

# Carregar o dataset
dados = load_dataset_from_file(datasetTxt)

# Verificar se carregou
if dados is None:
    print("❌ Falha ao carregar o dataset!")
else:
    print("✅ Dataset carregado com sucesso!\n")

    # DEBUG: Ver o conteúdo de cada seção
    print("=" * 50)
    print("📊 CONTEÚDO PARSEADO:")
    print("=" * 50)

    print("\n🎓 HEAD (Configurações):")
    print(dados['head'])

    print("\n📚 CC (Classes e Cursos):")
    for turma, cursos in dados['cc'].items():
        print(f"  {turma}: {cursos}")

    print("\n👨‍🏫 DSD (Professores e Cursos):")
    for prof, cursos in dados['dsd'].items():
        print(f"  {prof}: {cursos}")

    print("\n⏰ TR (Restrições de Tempo):")
    for prof, slots in dados['tr'].items():
        print(f"  {prof}: slots {slots}")

    print("\n🚪 RR (Restrições de Sala):")
    for curso, sala in dados['rr'].items():
        print(f"  {curso}: {sala}")

    print("\n💻 OC (Aulas Online):")
    for curso, indice in dados['oc'].items():
        print(f"  {curso}: aula {indice}")

    print("\n📝 OLW (Cursos com 1 aula/semana):")
    print(f"  {dados['olw']}")

✅ Dataset carregado com sucesso!

📊 CONTEÚDO PARSEADO:

🎓 HEAD (Configurações):
{}

📚 CC (Classes e Cursos):
  t01: ['UC11', 'UC12', 'UC13', 'UC14', 'UC15']
  t02: ['UC21', 'UC22', 'UC23', 'UC24', 'UC25']
  t03: ['UC31', 'UC32', 'UC33', 'UC34', 'UC35']

👨‍🏫 DSD (Professores e Cursos):
  jo: ['UC11', 'UC21', 'UC22', 'UC31']
  mike: ['UC12', 'UC23', 'UC32']
  rob: ['UC13', 'UC14', 'UC24', 'UC33']
  sue: ['UC15', 'UC25', 'UC34', 'UC35']

⏰ TR (Restrições de Tempo):
  mike: slots [13, 14, 15, 16, 17, 18, 19, 20]
  rob: slots [1, 2, 3, 4]
  sue: slots [9, 10, 11, 12, 17, 18, 19, 20]

🚪 RR (Restrições de Sala):
  UC14: Lab01
  UC22: Lab01

💻 OC (Aulas Online):
  UC21: aula 2
  UC31: aula 2

📝 OLW (Cursos com 1 aula/semana):
  []


In [6]:
# ============================
# 1) VARIÁVEIS (duas por UC)
# ============================
lesson_vars = []                      # ex: "t01_UC11_L1"
lesson_meta = {}                      # var -> {"class":..., "course":..., "lesson":1|2}
for turma, ucs in dados['cc'].items():
    for uc in ucs:
        for i in (1, 2):              # cada UC tem 2 aulas/semana
            var = f"{turma}_{uc}_L{i}"
            lesson_vars.append(var)
            lesson_meta[var] = {"class": turma, "course": uc, "lesson": i}

print(f"Total de variáveis criadas: {len(lesson_vars)}")


Total de variáveis criadas: 30


In [7]:
from constraint import Problem
from collections import Counter

# Criar o problema
problem = Problem()

# Adicionar todas as variáveis ao problema (domínio: blocos 1..20)
for v in lesson_vars:
    problem.addVariable(v, range(1, 21))  # 20 blocos possíveis

# Função auxiliar para descobrir o "dia" de um bloco
def dia_do_bloco(slot):
    return ((slot - 1) // 4) + 1   # blocos 1–4 = dia 1, 5–8 = dia 2, etc.

# Restrição: máximo 3 aulas por dia por turma
def max_3_por_dia(*slots):
    dias = [dia_do_bloco(s) for s in slots]  # converte blocos em dias
    contagem = Counter(dias)
    return all(qtd <= 3 for qtd in contagem.values())

# Aplicar a restrição a cada turma
for turma in dados['cc']:
    turma_vars = [v for v in lesson_vars if v.startswith(turma + "_")]
    problem.addConstraint(max_3_por_dia, turma_vars)


# ===============================================
# RESTRIÇÃO: Respeitar disponibilidade dos docentes
# ===============================================

# Vamos criar um mapa curso -> professor
curso_professor = {}
for prof, cursos in dados['dsd'].items():
    for c in cursos:
        curso_professor[c] = prof

# Agora aplicar as restrições
for prof, indisponiveis in dados['tr'].items():
    # Ver todas as UCs que este professor leciona
    cursos_prof = [c for c, p in curso_professor.items() if p == prof]
    
    # Para cada UC e para cada turma que tem essa UC
    for turma, ucs in dados['cc'].items():
        for uc in ucs:
            if uc in cursos_prof:
                # Para cada uma das 2 aulas dessa UC (L1 e L2)
                for i in (1, 2):
                    var = f"{turma}_{uc}_L{i}"
                    # Adicionar restrição: não pode estar em blocos indisponíveis
                    problem.addConstraint(
                        lambda slot, bloqueados=set(indisponiveis): slot not in bloqueados,
                        (var,)
                    )


# ===============================================
# RESTRIÇÃO: Aulas online (máx. 3 por dia)
# ===============================================

# Identificar as variáveis que são online
online_vars = []
for curso, li in dados['oc'].items():   # ex: UC21 -> aula 2
    for turma, ucs in dados['cc'].items():
        if curso in ucs:
            var = f"{turma}_{curso}_L{li}"
            online_vars.append(var)

print("Variáveis de aulas online:", online_vars)

# Função de restrição: máximo 3 online no mesmo dia
def max_3_online(*slots):
    dias = [dia_do_bloco(s) for s in slots]
    contagem = Counter(dias)
    return all(qtd <= 3 for qtd in contagem.values())

# Aplicar a restrição (somente se existirem aulas online)
if online_vars:
    problem.addConstraint(max_3_online, online_vars)

# ===============================================
# RESTRIÇÃO: Salas específicas para certas UCs
# ===============================================

# Criar um dicionário var -> sala fixa
salas_fixas = {}
for curso, sala in dados['rr'].items():  # ex: UC14 -> Lab01
    for turma, ucs in dados['cc'].items():
        if curso in ucs:
            for i in (1, 2):  # cada UC tem 2 aulas (L1, L2)
                var = f"{turma}_{curso}_L{i}"
                salas_fixas[var] = sala

print("Salas fixas atribuídas:", salas_fixas)


Variáveis de aulas online: ['t02_UC21_L2', 't03_UC31_L2']
Salas fixas atribuídas: {'t01_UC14_L1': 'Lab01', 't01_UC14_L2': 'Lab01', 't02_UC22_L1': 'Lab01', 't02_UC22_L2': 'Lab01'}


In [ ]:
# ============================
# SOFT: UCs em dias distintos
# ============================

solution = problem.getSolution()

if solution:
    for var, slot in sorted(solution.items()):
        print(f"{var:15s} -> bloco {slot:2d} (dia {dia_do_bloco(slot)})")
    print("✅ Solução viável encontrada!")
else:
    print("❌ Nenhuma solução encontrada.")

    
def soft_uc_dias_distintos(solution):
    violacoes = []  # lista de (turma, uc, dia, slot1, slot2)
    for turma, ucs in dados['cc'].items():
        for uc in ucs:
            v1 = f"{turma}_{uc}_L1"
            v2 = f"{turma}_{uc}_L2"
            s1, s2 = solution[v1], solution[v2]
            d1, d2 = dia_do_bloco(s1), dia_do_bloco(s2)
            if d1 == d2:  # violou a preferência
                violacoes.append((turma, uc, d1, s1, s2))
    return violacoes

violacoes = soft_uc_dias_distintos(solution)
print("\nSoft — UCs em dias distintos:")
if not violacoes:
    print("✔️ Todas as UCs têm as duas aulas em dias diferentes.")
else:
    print(f"⚠️ {len(violacoes)} UCs com as duas aulas no mesmo dia:")
    for turma, uc, dia, s1, s2 in violacoes:
        print(f"  {turma} {uc}: dia {dia} (slots {s1} e {s2})")


# ===============================================
# SOFT: Preferência por 4 dias de aulas por turma
# ===============================================
def soft_4_dias_por_turma(solution):
    penalizacoes = []  # lista de (turma, dias_usados)
    for turma in dados['cc']:
        # Pega todos os slots dessa turma
        turma_slots = [solution[v] for v in solution if v.startswith(turma + "_")]
        # Calcula os dias distintos
        dias_usados = {dia_do_bloco(s) for s in turma_slots}
        # Quantos dias tem aulas
        qtd_dias = len(dias_usados)
        # Penaliza se não forem exatamente 4
        if qtd_dias != 4:
            penalizacoes.append((turma, qtd_dias))
    return penalizacoes


# Executar a avaliação
penalizacoes = soft_4_dias_por_turma(solution)

print("\nSoft — Preferência por 4 dias de aulas por turma:")
if not penalizacoes:
    print("✔️ Todas as turmas têm aulas em exatamente 4 dias.")
else:
    print(f"⚠️ {len(penalizacoes)} turmas fora da preferência:")
    for turma, qtd in penalizacoes:
        print(f"  {turma}: tem aulas em {qtd} dias.")


# =======================================================
# SOFT: Preferência por aulas consecutivas no mesmo dia
# =======================================================
def soft_aulas_consecutivas(solution):
    violacoes = []  # lista de (turma, dia, blocos_usados)
    for turma in dados['cc']:
        # Pega todos os slots dessa turma
        turma_slots = [solution[v] for v in solution if v.startswith(turma + "_")]
        # Agrupa blocos por dia
        dias = {}
        for s in turma_slots:
            d = dia_do_bloco(s)
            dias.setdefault(d, []).append(s)
        
        # Verifica se os blocos de cada dia são consecutivos
        for dia, blocos in dias.items():
            blocos.sort()
            # Conta quantos "saltos" há entre blocos
            for i in range(len(blocos) - 1):
                if blocos[i+1] - blocos[i] != 1:
                    violacoes.append((turma, dia, blocos))
                    break  # só precisa de marcar 1 violação por dia
    return violacoes


# Executar a avaliação
violacoes = soft_aulas_consecutivas(solution)

print("\nSoft — Preferência por aulas consecutivas no mesmo dia:")
if not violacoes:
    print("✔️ Todas as turmas têm aulas consecutivas em cada dia.")
else:
    print(f"⚠️ {len(violacoes)} dias/turmas com aulas não consecutivas:")
    for turma, dia, blocos in violacoes:
        print(f"  {turma}: dia {dia} -> blocos {sorted(blocos)}")


# ===========================================================
# SOFT: Minimização do número de salas utilizadas por turma
# ===========================================================
def soft_min_salas_por_turma(solution, salas_fixas):
    penalizacoes = []  # lista de (turma, salas_usadas)
    
    for turma in dados['cc']:
        salas_usadas = set()
        
        # Percorre as aulas da turma
        for v in solution:
            if v.startswith(turma + "_"):
                # Se tiver sala fixa (no dicionário salas_fixas)
                if v in salas_fixas:
                    salas_usadas.add(salas_fixas[v])
                else:
                    # Se não tiver, podemos supor "SalaGenérica"
                    salas_usadas.add("SalaGenérica")
        
        qtd = len(salas_usadas)
        if qtd > 2:  # preferimos no máximo 2 salas por turma
            penalizacoes.append((turma, qtd, list(salas_usadas)))
    
    return penalizacoes


# Executar a avaliação
penalizacoes = soft_min_salas_por_turma(solution, salas_fixas)

print("\nSoft — Minimização do número de salas utilizadas por turma:")
if not penalizacoes:
    print("✔️ Todas as turmas usam no máximo 2 salas diferentes.")
else:
    print(f"⚠️ {len(penalizacoes)} turmas usam mais de 2 salas:")
    for turma, qtd, salas in penalizacoes:
        print(f"  {turma}: usa {qtd} salas -> {salas}")

